In [ ]:
%matplotlib inline
import numpy as np
import cv2
import os
import pandas as pd
from tqdm import tqdm
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
from google.colab.patches import cv2_imshow

In [ ]:
path1 = '/content/gdrive/My Drive/DATA/INMEYOK/output'
path2 = '/content/gdrive/My Drive/DATA/ISKEMI/output'
path3 = '/content/gdrive/My Drive/DATA/KANAMA/output'

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
image_list = []
label_list = []
def loaddata(path, label):
    all_images = os.listdir(path)
    i = 0
    for image in tqdm(all_images):
        img = cv2.imread(os.path.join(path, image))
        image_list.append(img)
        label_list.append(label)

In [ ]:
loaddata(path1,0)
loaddata(path2,1)
loaddata(path3,1)

  7%|▋         | 162/2229 [02:06<26:47,  1.29it/s]


KeyboardInterrupt: ignored

In [ ]:
len(image_list)
len(label_list)

In [ ]:
img = np.array(image_list)
lbl = np.array(label_list)

In [ ]:
del image_list
del label_list
import gc
gc.collect()

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(img,lbl, test_size=0.1,random_state=32, shuffle= True) 

In [ ]:
from sklearn.model_selection import KFold
num_folds = 6
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

In [ ]:
for layer in vgg_model.layers:
    layer.trainable = False
vgg_model.summary()

In [ ]:
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(X_train, y_train):
  modelVGG16 = Sequential()
  modelVGG16.add(vgg_model)
  modelVGG16.add(Flatten())
  modelVGG16.add(Dense(1024, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(512, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(256, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(128, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(64, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(32, activation='relu'))
  modelVGG16.add(Dropout(0.2))
  modelVGG16.add(Dense(1, activation='sigmoid'))

# Compile model 
  modelVGG16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                          tf.keras.metrics.Precision(),
                                                                          tf.keras.metrics.Recall(),
                                                                          tf.keras.metrics.AUC(from_logits=True)])
   # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = modelVGG16.fit(X_train[train], y_train[train], validation_split=0.1,batch_size=10, epochs=10)
   # Generate generalization metrics
  scores = modelVGG16.evaluate(X_train[test], y_train[test], verbose=0)
  print(f'Score for fold {fold_no}: {modelVGG16.metrics_names[0]} of {scores[0]}; {modelVGG16.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

In [ ]:
modelVGG16.save_weights('/content/gdrive/My Drive/VGG16')